In [ ]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.7 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=3c2461c62fc483e9376393b8dc5c42834dc376271f8fc2f7facf196985435077
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
import cv2
import os
from deepface import DeepFace

video_path = "/content/AN_WILTY_EP15_truth1.mp4"
output_folder = "frames"
os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
count, saved = 0, []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    if count % fps == 0:
        timestamp = int(count / fps)
        path = f"{output_folder}/frame_{timestamp}s.jpg"
        cv2.imwrite(path, frame)
        saved.append((path, timestamp))
    count += 1
cap.release()

print("Analyzing expressions...")
results = []
for path, ts in saved:
    try:
        analysis = DeepFace.analyze(img_path=path, actions=['emotion'], enforce_detection=False)
        results.append((ts, analysis[0]['dominant_emotion']))
    except Exception as e:
        results.append((ts, "Error"))

for ts, emotion in results:
    print(f"Time {ts}s: {emotion}")

print("\n--- Possible Deception Flags ---")
prev = None
for ts, emotion in results:
    if emotion in ['fear', 'surprise']:
        print(f"Suspicious emotion at {ts}s: {emotion}")
    if prev and emotion != prev:
        print(f"Emotion shift at {ts}s: {prev} → {emotion}")
    prev = emotion

Analyzing expressions...
Time 0s: neutral
Time 1s: neutral
Time 2s: neutral
Time 3s: sad
Time 4s: disgust

--- Possible Deception Flags ---
Emotion shift at 3s: neutral → sad
Emotion shift at 4s: sad → disgust


In [ ]:
import cv2
import os
from deepface import DeepFace

video_path = "/content/AN_WILTY_EP15_truth1.mp4"
frame_output_dir = "frames"
os.makedirs(frame_output_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = 0
frame_data = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_count % fps == 0:
        timestamp = int(frame_count / fps)
        frame_path = f"{frame_output_dir}/frame_{timestamp}s.jpg"
        cv2.imwrite(frame_path, frame)
        frame_data.append((timestamp, frame_path))
    frame_count += 1
cap.release()

emotion_sequence = []
for ts, path in frame_data:
    try:
        result = DeepFace.analyze(img_path=path, actions=['emotion'], enforce_detection=False)
        dominant_emotion = result[0]['dominant_emotion']
        emotion_sequence.append((ts, dominant_emotion))
    except:
        emotion_sequence.append((ts, "Error"))

deception_cues = {'fear', 'surprise', 'disgust'}
potential_lies = []
previous_emotion = None

for i, (ts, emotion) in enumerate(emotion_sequence):
    if emotion in deception_cues:
        potential_lies.append(ts)
    elif previous_emotion and emotion != previous_emotion:
        potential_lies.append(ts)
    previous_emotion = emotion

potential_lies = sorted(set(potential_lies))

print("Potential lie detected at seconds:")
for ts in potential_lies:
    print(f"{ts}s")


Potential lie detected at seconds:
3s
4s


## 🧠 Prediction with Confidence Scores (Probability Format)
This section loads each video from the dataset folder, performs inference, and outputs the predicted label along with the confidence score.

In [ ]:

import os
import torch
import torch.nn.functional as F
import pandas as pd
from torchvision import transforms
import cv2
from model import EmotionModel  # Assuming model class name is EmotionModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionModel()
model.load_state_dict(torch.load("model.pth", map_location=device))  # Adjust path if needed
model.eval().to(device)

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def extract_middle_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames // 2)
    ret, frame = cap.read()
    cap.release()
    return frame if ret else None

video_results = []
label_map = {0: "deceptive", 1: "truthful"}

for label_folder in ["truthful", "deceptive"]:
    folder_path = os.path.join("dataset", label_folder)
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            path = os.path.join(folder_path, filename)
            frame = extract_middle_frame(path)
            if frame is None:
                continue
            input_tensor = transform(frame).unsqueeze(0).to(device)
            with torch.no_grad():
                output = model(input_tensor)
                probs = F.softmax(output, dim=1)
                confidence, pred_class = torch.max(probs, dim=1)
                predicted_label = label_map[pred_class.item()]
                video_results.append({
                    "Filename": filename,
                    "Predicted Label": predicted_label,
                    "Confidence": round(confidence.item(), 4)
                })

df = pd.DataFrame(video_results)
import ace_tools as tools; tools.display_dataframe_to_user(name="Video Predictions", dataframe=df)
